# Capital Asset Pricing Model
$$ER_{i} = R_{f} + \beta_{i}(ER_{m} - R_{f})$$

$ER$ = Expected Return of Asset \
$ER_{m}$ = Expected Return of Market \
$R_{f}$ = Risk-free rate \
$\beta$ = Beta of the Investment (This is a relative risk measurement with respect to the market) \
$(ER_{m} - R_{f})$ = Market Risk Premium

In [1]:
import re
from io import StringIO
from datetime import datetime, timedelta
import requests
import pandas as pd
import numpy as np
import pandas_datareader as pdr
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
# Loading in our data
def get_historical_Data(tickers):
    """
    This function returns a pd dataframe with all of the adjusted closing information
    """
    data = pd.DataFrame()
    names = list()
    for i in tickers:
        data = pd.concat([data,pdr.get_data_yahoo(symbols=i, start=datetime(2010, 11, 17), end=datetime(2020, 11, 17)).iloc[:,5]], axis = 1)
        names.append(i)
    data.columns = names
    return data

In [3]:
# Reading in our 10 year Treasury Constant Maturity Rate.
# https://fred.stlouisfed.org/series/DGS10
T_rate = pd.read_csv('DGS10.csv')

In [4]:
T_rate['DATE'] = pd.to_datetime(T_rate['DATE'], format = "%Y-%m-%d").dt.strftime('%m-%d-%Y')

In [5]:
T_rate

,DATE,DGS10
0,11-15-2010,2.92
1,11-16-2010,2.85
2,11-17-2010,2.89
3,11-18-2010,2.90
4,11-19-2010,2.88
...,...,...
2604,11-06-2020,0.83
2605,11-09-2020,0.96
2606,11-10-2020,0.98
2607,11-11-2020,.


In [6]:
count = 0
for i in T_rate['DGS10']:
    if i =='.':
        T_rate['DGS10'][count] = T_rate['DGS10'][count-1]
    count+=1

In [7]:
#T_rate

In [8]:
#    The ticker names of the companies that we will be looking at. (And the S&P500)
ticks = ["DPZ", '^GSPC']
d = get_historical_Data(ticks)

In [9]:
print(d.shape)
#

(2518, 2)


In [10]:
d = d.reset_index()
for i in range(d.shape[0]):
    mo = ''
    da = ''
    if d['index'][i].month < 10:
        mo = '0' + str(d['index'][i].month)
    else:
        mo = str(d['index'][i].month)
    if d['index'][i].day < 10:
        da = '0' + str(d['index'][i].day)
    else:
        da = str(d['index'][i].day)
    d['index'][i] = mo + '-' + da + '-' + str(d['index'][i].year) 
# Changing the index name to date
d = d.rename(columns = {"index": "DATE"})

<ipython-input-10-7113205fa697>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['index'][i] = mo + '-' + da + '-' + str(d['index'][i].year)
C:\Users\Spenc\anaconda3\envs\finance\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [11]:
d

,DATE,DPZ,^GSPC
0,11-17-2010,12.098056,1178.589966
1,11-18-2010,12.276972,1196.689941
2,11-19-2010,12.259933,1199.729980
3,11-22-2010,12.549608,1197.839966
4,11-23-2010,12.447366,1180.729980
...,...,...,...
2513,11-11-2020,394.670013,3572.659912
2514,11-12-2020,390.549988,3537.010010
2515,11-13-2020,389.600006,3585.149902
2516,11-16-2020,384.820007,3626.909912


In [12]:
# Merge dataframes
data = pd.merge(left = T_rate, right = d, left_on = 'DATE', right_on = 'DATE')
data = data.rename(columns = {"DGS10": "10_Year_Rate"})

In [13]:
data

,DATE,10_Year_Rate,DPZ,^GSPC
0,11-17-2010,2.89,12.098056,1178.589966
1,11-18-2010,2.90,12.276972,1196.689941
2,11-19-2010,2.88,12.259933,1199.729980
3,11-22-2010,2.80,12.549608,1197.839966
4,11-23-2010,2.77,12.447366,1180.729980
...,...,...,...,...
2510,11-06-2020,0.83,399.850006,3509.439941
2511,11-09-2020,0.96,376.549988,3550.500000
2512,11-10-2020,0.98,381.670013,3545.530029
2513,11-11-2020,0.98,394.670013,3572.659912


In [14]:
# Because of the CAPM formula, we need to calculate the Percent changes of our given assets.
data['DPZ_Daily_Returns'] = data['DPZ'].pct_change()
data['SP500_Daily_Returns'] = data['^GSPC'].pct_change()

In [15]:
data

,DATE,10_Year_Rate,DPZ,^GSPC,DPZ_Daily_Returns,SP500_Daily_Returns
0,11-17-2010,2.89,12.098056,1178.589966,NaN,NaN
1,11-18-2010,2.90,12.276972,1196.689941,0.014789,0.015357
2,11-19-2010,2.88,12.259933,1199.729980,-0.001388,0.002540
3,11-22-2010,2.80,12.549608,1197.839966,0.023628,-0.001575
4,11-23-2010,2.77,12.447366,1180.729980,-0.008147,-0.014284
...,...,...,...,...,...,...
2510,11-06-2020,0.83,399.850006,3509.439941,0.009162,-0.000288
2511,11-09-2020,0.96,376.549988,3550.500000,-0.058272,0.011700
2512,11-10-2020,0.98,381.670013,3545.530029,0.013597,-0.001400
2513,11-11-2020,0.98,394.670013,3572.659912,0.034061,0.007652


In [16]:
# Taking out the first row as it is NaN
data = data[1:]

# Calculate the excess return

data['Excess_Return_DPZ'] = data['DPZ_Daily_Returns'] - data['10_Year_Rate'].astype('float') / 100
data['Excess_Return_SP500'] = data['SP500_Daily_Returns'] - data['10_Year_Rate'].astype('float') / 100

<ipython-input-16-9c77a99dd263>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Excess_Return_DPZ'] = data['DPZ_Daily_Returns'] - data['10_Year_Rate'].astype('float') / 100
<ipython-input-16-9c77a99dd263>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Excess_Return_SP500'] = data['SP500_Daily_Returns'] - data['10_Year_Rate'].astype('float') / 100


In [17]:
data

,DATE,10_Year_Rate,DPZ,^GSPC,DPZ_Daily_Returns,SP500_Daily_Returns,Excess_Return_DPZ,Excess_Return_SP500
1,11-18-2010,2.90,12.276972,1196.689941,0.014789,0.015357,-0.014211,-0.013643
2,11-19-2010,2.88,12.259933,1199.729980,-0.001388,0.002540,-0.030188,-0.026260
3,11-22-2010,2.80,12.549608,1197.839966,0.023628,-0.001575,-0.004372,-0.029575
4,11-23-2010,2.77,12.447366,1180.729980,-0.008147,-0.014284,-0.035847,-0.041984
5,11-24-2010,2.93,12.694441,1198.349976,0.019850,0.014923,-0.009450,-0.014377
...,...,...,...,...,...,...,...,...
2510,11-06-2020,0.83,399.850006,3509.439941,0.009162,-0.000288,0.000862,-0.008588
2511,11-09-2020,0.96,376.549988,3550.500000,-0.058272,0.011700,-0.067872,0.002100
2512,11-10-2020,0.98,381.670013,3545.530029,0.013597,-0.001400,0.003797,-0.011200
2513,11-11-2020,0.98,394.670013,3572.659912,0.034061,0.007652,0.024261,-0.002148


In [18]:
# Running a regression to calculate Beta
results = smf.ols('Excess_Return_DPZ ~ Excess_Return_SP500', data = data).fit()

In [19]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:      Excess_Return_DPZ   R-squared:                       0.246
Model:                            OLS   Adj. R-squared:                  0.246
Method:                 Least Squares   F-statistic:                     818.9
Date:                Tue, 17 Nov 2020   Prob (F-statistic):          3.96e-156
Time:                        22:15:15   Log-Likelihood:                 6711.5
No. Observations:                2514   AIC:                        -1.342e+04
Df Residuals:                    2512   BIC:                        -1.341e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -0.0042    

In [20]:
# Beta from the OLS above
Beta = 1.1367
# Once we have Beta, we can calculate the expected return of the company BASED on the market.
average_risk_free_rate = data['10_Year_Rate'].astype('float').mean() /100
# Using the Historical Rate of return for the S&P500 market...
# Including dividends but not accounting for inflation
average_return_SP500 = (1.311 * (1-.0441) * (1.2194) * (1.1193) * (1.0131) * (1.1381))**(1/5) - 1

print('Average Risk Free Rate',average_risk_free_rate)
print('Average Return S&P500',average_return_SP500)
print('Expected Return of DPZ',average_risk_free_rate + Beta * (average_return_SP500 - average_risk_free_rate))

Average Risk Free Rate 0.021997931583134445
Average Return S&P500 0.14548159954598372
Expected Return of DPZ 0.16236181695650523


# Doing the Same thing but for the monthly data.

In [13]:
# Using Monthly data for T Rates
mT_rate = pd.read_csv('Monthly_DGS10.csv')
#https://fred.stlouisfed.org/series/DGS10
count = 0
for i in mT_rate['DGS10']:
    if i =='.':
        mT_rate['DGS10'][count] = mT_rate['DGS10'][count-1]
    count+=1
mT_rate

,DATE,DGS10
0,2015-11-01,2.263158
1,2015-12-01,2.242727
2,2016-01-01,2.085263
3,2016-02-01,1.779500
4,2016-03-01,1.889091
5,2016-04-01,1.805238
6,2016-05-01,1.806190
7,2016-06-01,1.644091
8,2016-07-01,1.504000
9,2016-08-01,1.556522


In [6]:
import pandas_datareader.data as web
# Monthly Company data
tick = 'XOM'
Mdata = web.get_data_yahoo(tick,'11/24/2015',interval='m')
#Monthly Market data
MMdata = web.get_data_yahoo('^GSPC','11/24/2015',interval='m')

In [7]:
Mdata = Mdata.rename(columns = {"Adj Close": tick})
MMdata = MMdata.rename(columns = {"Adj Close": "SP500"})

In [8]:
Mdata = pd.concat([Mdata[tick], MMdata['SP500'] ], axis = 1)

In [9]:
Mdata

,XOM,SP500
Date,,
2015-12-01,61.545231,2043.939941
2016-01-01,61.466263,1940.239990
2016-02-01,63.282223,1932.229980
2016-03-01,66.597282,2059.739990
2016-04-01,70.429482,2065.300049
...,...,...
2020-08-01,38.244602,3500.310059
2020-09-01,33.521252,3363.000000
2020-10-01,31.851534,3269.959961


In [10]:
Mdata = Mdata.reset_index()
for i in range(Mdata.shape[0]):
    mo = ''
    da = ''
    if Mdata['Date'][i].month < 10:
        mo = '0' + str(Mdata['Date'][i].month)
    else:
        mo = str(Mdata['Date'][i].month)
        
    if Mdata['Date'][i].day < 10:
        da = '0' + str(Mdata['Date'][i].day)
    else:
        da = str(Mdata['Date'][i].day)
    Mdata['Date'][i] = str(Mdata['Date'][i].year) + '-' + mo  + '-' +  da
# Changing the index name to date
Mdata = Mdata.rename(columns = {"Date": "DATE"})
Mdata

<ipython-input-10-3c47a790871d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Mdata['Date'][i] = str(Mdata['Date'][i].year) + '-' + mo  + '-' +  da
C:\Users\Spenc\anaconda3\envs\finance\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,DATE,XOM,SP500
0,2015-12-01,61.545231,2043.939941
1,2016-01-01,61.466263,1940.239990
2,2016-02-01,63.282223,1932.229980
3,2016-03-01,66.597282,2059.739990
4,2016-04-01,70.429482,2065.300049
...,...,...,...
56,2020-08-01,38.244602,3500.310059
57,2020-09-01,33.521252,3363.000000
58,2020-10-01,31.851534,3269.959961
59,2020-11-01,38.432755,3577.590088


In [11]:
Mdata['SP500_Daily_Returns'] = Mdata['SP500'].pct_change()
Mdata['{}_Daily_Returns'.format(tick)] = Mdata[tick].pct_change()
Mdata = Mdata[1:]

In [14]:
# Merging Data Frames
Mdata = pd.merge(left = mT_rate, right = Mdata, left_on = 'DATE', right_on = 'DATE')
Mdata

,DATE,DGS10,XOM,SP500,SP500_Daily_Returns,XOM_Daily_Returns
0,2016-01-01,2.085263,61.466263,1940.239990,-0.050735,-0.001283
1,2016-02-01,1.779500,63.282223,1932.229980,-0.004128,0.029544
2,2016-03-01,1.889091,66.597282,2059.739990,0.065991,0.052385
3,2016-04-01,1.805238,70.429482,2065.300049,0.002699,0.057543
4,2016-05-01,1.806190,70.923447,2096.949951,0.015325,0.007014
5,2016-06-01,1.644091,75.311615,2098.860107,0.000911,0.061872
6,2016-07-01,1.504000,71.463280,2173.600098,0.035610,-0.051099
7,2016-08-01,1.556522,70.009094,2170.949951,-0.001219,-0.020349
8,2016-09-01,1.630476,70.719543,2168.270020,-0.001234,0.010148
9,2016-10-01,1.764500,67.510887,2126.149902,-0.019426,-0.045372


In [15]:
Mdata['Excess_Return_{}'.format(tick)] = Mdata['{}_Daily_Returns'.format(tick)] - Mdata['DGS10'].astype('float') / 100
Mdata['Excess_Return_SP500'] = Mdata['SP500_Daily_Returns'] - Mdata['DGS10'].astype('float') / 100
Mdata

,DATE,DGS10,XOM,SP500,SP500_Daily_Returns,XOM_Daily_Returns,Excess_Return_XOM,Excess_Return_SP500
0,2016-01-01,2.085263,61.466263,1940.239990,-0.050735,-0.001283,-0.022136,-0.071588
1,2016-02-01,1.779500,63.282223,1932.229980,-0.004128,0.029544,0.011749,-0.021923
2,2016-03-01,1.889091,66.597282,2059.739990,0.065991,0.052385,0.033494,0.047100
3,2016-04-01,1.805238,70.429482,2065.300049,0.002699,0.057543,0.039490,-0.015353
4,2016-05-01,1.806190,70.923447,2096.949951,0.015325,0.007014,-0.011048,-0.002737
5,2016-06-01,1.644091,75.311615,2098.860107,0.000911,0.061872,0.045431,-0.015530
6,2016-07-01,1.504000,71.463280,2173.600098,0.035610,-0.051099,-0.066139,0.020570
7,2016-08-01,1.556522,70.009094,2170.949951,-0.001219,-0.020349,-0.035914,-0.016784
8,2016-09-01,1.630476,70.719543,2168.270020,-0.001234,0.010148,-0.006157,-0.017539
9,2016-10-01,1.764500,67.510887,2126.149902,-0.019426,-0.045372,-0.063017,-0.037071


In [16]:
# Running a regression to calculate Beta
# Can play around with what risk free rate to use to get your relative beta.
results = smf.ols('Excess_Return_{} ~ Excess_Return_SP500'.format(tick), data = Mdata).fit()

In [17]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:      Excess_Return_XOM   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.509
Method:                 Least Squares   F-statistic:                     60.09
Date:                Tue, 24 Nov 2020   Prob (F-statistic):           1.98e-10
Time:                        14:34:00   Log-Likelihood:                 91.118
No. Observations:                  58   AIC:                            -178.2
Df Residuals:                      56   BIC:                            -174.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -0.0153    

In [18]:
# Beta from the OLS above
Beta = 1.0775
# Once we have Beta, we can calculate the expected return of the company BASED on the market.
average_risk_free_rate = Mdata['DGS10'].astype('float').mean() /100
# Using the Historical Rate of return for the S&P500 market...
# Including dividends but not accounting for inflation
average_return_SP500 = (1.311 * (1-.0441) * (1.2194) * (1.1193) * (1.0131) * (1.1381))**(1/5) - 1

print('Average Risk Free Rate',average_risk_free_rate)
print('Average Return S&P500',average_return_SP500)
print('Expected Return of {}'.format(tick),average_risk_free_rate + Beta * (average_return_SP500 - average_risk_free_rate))

Average Risk Free Rate 0.02063015824879768
Average Return S&P500 0.14548159954598372
Expected Return of XOM 0.15515758624651563
